### 1 - Import libs

**Execute only**

In [ ]:
import time
from datetime import datetime as dt

import ee
from sepal_ui.mapping import SepalMap

# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

### 2 - Basic Input Variables

**DEFINE YOUR INPUT VARIABLES**

In [ ]:
# Area of Interest
country = 'Kenya'   # country case
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country)) # here any feature collection can be selected

# Grid Size (in degrees)
grid_size = 1

# define an output file name for CEO 
# don't forget the .csv extention
# IF NOT DEFINED; will go into module_results
# outfile = "sbae.csv"

### 3 - Generate GRID and POINTS

**Execute only**

In [ ]:
# Generate the grid
grid, grid_fc = h.generate_grid(aoi, grid_size, grid_size, 0, 0)

# get a random point over each grid cell
randomInCell = grid_fc.map(h.get_random_point).filterBounds(aoi)

# get the centroid of each gridcell
centerInCell = grid_fc.map(h.get_center_point).filterBounds(aoi)

# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = imageCollection.filterBounds(aoi).first().clip(aoi)

stratifiedSamples = esaLc.stratifiedSample(**{
    "numPoints": 5,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.set_id)

### 4 - Display in map

**Execute only**

In [ ]:
# intialize Map
Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

# add layers
Map.addLayer(ee.Image().byte().paint(featureCollection=aoi,color=1,width=3), {"palette": "blue"}, 'Area of interest')
Map.addLayer(esaLc, False, 'ESA Worldcover 2020')
Map.addLayer(stratifiedSamples.draw(color="black", pointRadius=2), {}, 'Stratified Samples per class from ESA LC')
Map.addLayer(ee.Image().byte().paint(featureCollection=grid,color=1,width=3), {"palette": "black"}, 'Grid')
Map.addLayer(randomInCell.draw(color="purple", pointRadius=2), {}, '1 Center Sample per gridcell (Systematic)')
Map.addLayer(centerInCell.draw(color="white", pointRadius=2), {}, '1 Random Sample per gridcell (Systematic random)')
#
Map

### 5 - Add auxillary data from global datasets

**Execute only**

In [ ]:
point_id = 'point_id'
point_with_global_products = h.sample_global_products(centerInCell, point_id)
point_with_global_products